In [24]:
import pandas as pd

In [25]:
maestra_path = 'C:/Users/Rodrigo/Desktop/Roxana Files/Scopus/maestra.xlsx'
scopus_path = 'C:/Users/Rodrigo/Desktop/Roxana Files/Scopus/scopus_resultados_beta.xlsx'

df_maestra = pd.read_excel(maestra_path, sheet_name='BaseMaestra')
df_scopus = pd.read_excel(scopus_path)

df_scopus.shape

(23179, 14)

In [26]:
df_scopus_chile = df_scopus[df_scopus['pais_afiliacion'].str.upper() == 'CHILE']
df_scopus_chile = df_scopus_chile.drop_duplicates(subset=['nombre_completo_scopus', 'afiliacion_actual', 'scopus_id'])

df_scopus_chile.shape


(2273, 14)

In [27]:
def normalizar_columnas(df, columnas):
    for col in columnas:
        df[col] = df[col].str.upper().str.strip().fillna('')
    return df


In [35]:
df_maestra = normalizar_columnas(df_maestra, ['Primer_Nombre', 'Segundo_Nombre', 'Apellido_Paterno', 'Apellido_Materno'])
df_scopus_chile = normalizar_columnas(df_scopus_chile, ['nombre_completo_scopus'])

df_maestra['nombre_completo_maestra'] = df_maestra['Primer_Nombre'] + ' ' + df_maestra['Segundo_Nombre'] + ' ' + df_maestra['Apellido_Paterno'] + ' ' + df_maestra['Apellido_Materno']
df_maestra['nombre_completo_maestra'] = df_maestra['nombre_completo_maestra'].str.strip()


In [29]:
def generar_combinaciones(row):
    combinaciones = [
        f"{row['Primer_Nombre']} {row['Apellido_Paterno']}",
        f"{row['Primer_Nombre']} {row['Apellido_Materno']}",
        f"{row['Segundo_Nombre']} {row['Apellido_Paterno']}",
        f"{row['Segundo_Nombre']} {row['Apellido_Materno']}",
        f"{row['Primer_Nombre']} {row['Segundo_Nombre']} {row['Apellido_Paterno']}",
        f"{row['Primer_Nombre']} {row['Segundo_Nombre']} {row['Apellido_Materno']}"
    ]
    return combinaciones


In [30]:
df_maestra['combinaciones_nombres'] = df_maestra.apply(generar_combinaciones, axis=1)

In [31]:
def buscar_coincidencias(fila_maestra, df_scopus):
    coincidencias = df_scopus[df_scopus['nombre_completo_scopus'].apply(
        lambda x: any([combinacion in x for combinacion in fila_maestra['combinaciones_nombres']])
    )]
    return coincidencias



In [32]:
nuevas_columnas = ['scopus_id', 'numero_publicaciones', 'cited_by_count', 'citation_count', 'afiliacion_actual']
for col in nuevas_columnas:
    df_maestra[col] = None 

In [33]:
for index, row in df_maestra.iterrows():
    coincidencias = buscar_coincidencias(row, df_scopus_chile)
    if not coincidencias.empty:
        # Si hay coincidencias, tomamos la primera y actualizamos la fila de la base maestra
        primera_coincidencia = coincidencias.iloc[0]
        df_maestra.at[index, 'scopus_id'] = primera_coincidencia['scopus_id']
        df_maestra.at[index, 'numero_publicaciones'] = primera_coincidencia['numero_publicaciones']
        df_maestra.at[index, 'cited_by_count'] = primera_coincidencia['cited_by_count']
        df_maestra.at[index, 'citation_count'] = primera_coincidencia['citation_count']
        df_maestra.at[index, 'afiliacion_actual'] = primera_coincidencia['afiliacion_actual']


In [34]:
output_path = 'C:/Users/Rodrigo/Desktop/Roxana Files/Scopus/match_resultados.xlsx'
df_maestra.to_excel(output_path, index=False)

print("Actualización completada. Archivo guardado en:", output_path)

Actualización completada. Archivo guardado en: C:/Users/Rodrigo/Desktop/Roxana Files/Scopus/match_resultados.xlsx
